

---

# Introduction

In this post, we’ll explore how to implement each component of Llama 3 entirely in pure JAX. I grew tired of writing the code in torch, so I decided to switch to JAX—a library that works much like NumPy but offers powerful features such as `AutoDiff`, `jit` and `vmap` which are very helpful for training neural networks.

I began this project a few months ago and hit some errors along the way that forced me to put it on hold. Recently, I came across on this [blog post](https://alessiodevoto.github.io/ViT-in-pure-JAX/#the-model-is-just-a-function) where the author implemented a Vision Transformer in pure JAX. That post motivate me, and I finally completed my project. In this blog, I share my journey and detail how I built Llama 3 using pure JAX.

---


In [ ]:
The above figure is of llama architetue. we will bw coverin each part from above image like :

- implemting complete model in jax 
- training it on shakespaere data with 125M size model 
- Ineference 

In [ ]:
## Tokenization 

- When we canont raw text to the model directly cause model don't undxerstnad our languase so we need to convert it into numbers. 
Tokenization is process in which we divide the text into words and subwords knows as token. But this is not effiient it requires more memory so there some advanced 
Tokenization method that we going to use like BPE(this was used in trianing llama3). I will not cover this you can checkout this great explantion of bpe(byte pair encoding) by andrej karpathy. 

In [ ]:
import jax.numpy as jnp
import tiktoken

# Load GPT-2 BPE encoding
enc = tiktoken.get_encoding("gpt2") 

# reading a line from 
with open('../shakespeare.txt', 'r') as f:
    text = f.readlines()[0]  # Take the first line

# Encode the text into token IDs
tokens = enc.encode(text)
data = jnp.array(tokens, dtype=jnp.int32)  # Store as JAX array

# Decode back to text
decoded_text = enc.decode(tokens)


print("original Text:", text.strip())
print("encoded Tokens:", tokens)
print("decoded Text:", decoded_text)

## Ouput ##

# Original Text: From fairest creatures we desire increase,
# Encoded Tokens: [220, 3574, 37063, 301, 8109, 356, 6227, 2620, 11, 198]
# Decoded Text:   From fairest creatures we desire increase,


Original Text: From fairest creatures we desire increase,
Encoded Tokens: [220, 3574, 37063, 301, 8109, 356, 6227, 2620, 11, 198]
Decoded Text:   From fairest creatures we desire increase,



## Embeddings 

- We need cause we cannot provide the tokens directly cause tokens are descreate but neural networks operate on continuous numerical data. This also helps in mathematical operations (addition, multiplication, etc.). Embedding also help in encoding semantic and syntactic relationships between tokens.Similar tokens (e.g., "cat" and "dog") are mapped to nearby points in the embedding space. Relationships like analogies (e.g., "king" – "man" + "woman" ≈ "queen") can emerge in the vector space. The model learns these patterns during training, enabling it to generalize to unseen text.

- LLM use the dynamic embeding which can be updated. Initial embeddings represent the "identity" of a token (e.g., the word "bank"). Subsequent layers (attention mechanisms) refine these embeddings to incorporate context. For example:
"Bank" in "river bank" vs. "bank account" gets different contextualized representations.